In [124]:
## https://www.10000recipe.com/recipe/list.html
## 만개의 레시피

In [1]:
## Anaconda - pip install PyMySQL
import pymysql

## 연속적 print, output clear
## source : display(target), clear_output(wait=True)
from IPython.display import clear_output

#Console Prograss Bar
from tqdm import tqdm, trange, tqdm_notebook

from time import sleep

## DataFrame 0
import numpy as np 
# Array
import pandas as pd

## http Requests
import requests
from bs4 import BeautifulSoup

In [2]:
## 스크래핑 url
target_url = {
    'domain'     : 'https://www.10000recipe.com',  # Target Domain  
    'recipe_list': '/recipe/list.html',            # Target page url
    'recipe'     : ''                              # Tager url
}

## 스크래핑 url 에 보내질 params
page_params = {
    'order'      : 'reco',     # 정렬 
    'cat4'       : '66' ,      # 분류
    'cat4_name'  : '빵',   # 분류 이름
    'page'       : 1,          # Page Number
    'sequence'   : 0           # Recipe sequence
}   

In [3]:
## Csv add Save..
#  param_dict ex : {'key': '레시피', 'kind' : '밑반찬}
def fn_save_csv(param_dict):
    
    #dict를 리스트로 변환 후 DataFrame 으로 변환.
    df_save_data = pd.DataFrame([param_dict])
    
    if(param_dict.get('page_num') == 1 and param_dict.get('sequence') == 1):
        #to_csv(mode a : 기존 내역에서 추가로 입력한다. , index=False : 인덱스 입력을 x, header=True컬럼명을 입력한다)
        df_save_data.to_csv('recipe.csv', mode='a', index=False, header=True)
    else:
        #to_csv(mode a : 기존 내역에서 추가로 입력한다. , index=False : 인덱스 입력을 x, header=True컬럼명을 입력하지 않는다.)
        df_save_data.to_csv('recipe.csv', mode='a', index=False, header=False)

In [4]:
## Function MySql Area...........
#----------------------------------------
#       DDL db tool 에서 처리 
#       Table Schema
#----------------------------------------

####### recipe : 요리 정보
#drop table if exists recipe;
#CREATE TABLE IF NOT EXISTS recipe(
#    idx          INT(11)       NOT NULL AUTO_INCREMENT PRIMARY KEY,     # INDEX NO [PK]
#    page_num     INT(11)       NOT NULL,                                # page 번호
#    cooking_type VARCHAR(100)  NOT NULL,                                # 요리 종류
#    title        VARCHAR(100)  NOT NULL,                                # 제목
#    ingredient   VARCHAR(1000),                                         # 재료
#    sorce        VARCHAR(1000),                                         # 재료 외 양념장 등
#    uri          VARCHAR(500)  NOT null,                                # URL
#    insert_dt    datetime                 default NOW()                 # 저장일자 시분초
#)  ENGINE=InnoDB DEFAULT CHARSET=utf8
#;
#
#drop table if exists scraping_error;
#create table if not exists scraping_error(
#    idx          INT(11)       NOT NULL AUTO_INCREMENT PRIMARY KEY,     # INDEX NO [PK]
#    page_num     INT(11),                                               # page 번호
#    cooking_type VARCHAR(100),                                          # 요리 종류
#    title        VARCHAR(100),                                          # 제목
#    error_code   VARCHAR(100),                                          # response error code
#    error_str    VARCHAR(2000),                                         # response error str
#    uri          VARCHAR(500),                              # URL
#    insert_dt    datetime                 default NOW()                 # 저장일자 시분초
#)  ENGINE=InnoDB DEFAULT CHARSET=utf8 
#;

#################################################
# Database Insert Proc
#################################################
def fn_insert_sql(params):

    #----------------------------------------
    # Database(Mysql) Connect Info
    #----------------------------------------
    
    # mysql 연결을 위한 정보를 저장하고 있는 mysql Connect 객체 생성한다.
    # host : ip위치(localhost : 현재 pc)
    # port : mysql port 번호
    # db   : 데이터베이스 이름
    # user : 데이터베이스 접근 login Id
    # pw   : 데이터베이스 접근 login Id의 password
    # charset : character Set
    # autocommit = True : Insert, Updaete, Delete 등이 자동 Commit 된다.
    # pymysql.cursors.DictCursor : 결과값을 Dict(Key : Value)로 Return한다.
    
    mysql_conn = pymysql.connect(
        host='localhost', 
        port=21500,
        db='mydb', user='root', password='',
        charset='utf8', autocommit=True,
        cursorclass=pymysql.cursors.DictCursor
    )
    
    #return DataFrame
    rtn = pd.DataFrame;
    try:
        #DataBase Sql (Structured Query Language)
        cursor = mysql_conn.cursor()
        
        ins_sql = '''
            INSERT INTO recipe (
                page_num,
                seq,
                cooking_type, 
                title, 
                ingredient, 
                sorce,
                uri
            )
            VALUES (
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s
            )
        '''
        
        # 이 때에 corsor에서 database와 연결 및 Query를 보내고, returen 값을 받는다.
        # 실제 이때 연결된다 볼 수 있다.
        cursor.execute(ins_sql, (
            params.get('page_num'),
            params.get('sequence'),
            params.get('cooking_type'),
            params.get('title'),
            params.get('ingredient'),
            params.get('source'),
            params.get('uri')
        ))
        rtn = cursor.fetchall()
#    except:

    finally:
        #Database 연결 해제
        mysql_conn.close()
    
    return rtn;

def fn_select_sql():

    #----------------------------------------
    # Database(Mysql) Connect Info
    #----------------------------------------
    
    # mysql 연결을 위한 정보를 저장하고 있는 mysql Connect 객체 생성한다.
    # host : ip위치(localhost : 현재 pc)
    # port : mysql port 번호
    # db   : 데이터베이스 이름
    # user : 데이터베이스 접근 login Id
    # pw   : 데이터베이스 접근 login Id의 password
    # charset : character Set
    # autocommit = True : Insert, Updaete, Delete 등이 자동 Commit 된다.
    # pymysql.cursors.DictCursor : 결과값을 Dict(Key : Value)로 Return한다.
    mysql_conn = pymysql.connect(
        host='localhost', 
        port=21500,
        db='mydb', user='root', password='',
        charset='utf8', autocommit=True,
        cursorclass=pymysql.cursors.DictCursor
    )
    
    rtn = pd.DataFrame;
    try:
        
        #DataBase Sql (Structured Query Language)
        cursor = mysql_conn.cursor()
        
        ins_sql = '''
            SELECT * FROM recipe
        '''
    
        cursor.execute(ins_sql)
        rtn = cursor.fetchall()
#    except:

    finally:
        #Database 연결 해제
        mysql_conn.close()
    
    return rtn;

In [6]:
df = pd.DataFrame(fn_select_sql())
df.head(5)

,cooking_type,idx,ingredient,insert_dt,page_num,seq,sorce,title,uri
0,밑반찬,1,"새송이버섯,버터,어린잎채소",2021-10-06 18:23:56,1,1,"간장,맛술,올리고당",버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,https://www.10000recipe.com/recipe/6912734
1,밑반찬,2,"팽이버섯,청양고추,식용유,다진마늘",2021-10-06 18:23:57,1,2,"고추가루,설탕,미림,진간장,굴소스,고추장,된장,물",500원 이면 밥한끼~ 뚝딱하는 팽이버섯조림~ 너무 맛있어용^^~,https://www.10000recipe.com/recipe/6956266
2,밑반찬,3,"건새우,물,마늘종,소금",2021-10-06 18:23:57,1,3,"설탕,간장,식용유,참기름,물엿,통깨",[마늘종볶음]단짠단짠 자꾸만 손이가요~,https://www.10000recipe.com/recipe/6957535
3,밑반찬,4,"베이컨,대파,떡",2021-10-06 18:23:58,1,4,"간장,참기름,물엿,기름",달콤 짭조름♡ 떡 베이컨 간장조림♡,https://www.10000recipe.com/recipe/6843136
4,밑반찬,5,풋고추,2021-10-06 18:23:58,1,5,"된장,다진 마늘,송송썰은 대파,참기름,통깨,매실원액,설탕",아삭아삭한 고추된장박이,https://www.10000recipe.com/recipe/4030952


In [7]:
#html Document를 Return 받는 Function
def fn_get_html(url, params={}):
    ## Get html Doc 
    
    ## navigator.userAgent 크롬 개발자 도구(consol 입력)에서 확인.
    ## 현재 요청을 보내는 곳은 내 pc인걸 확인 시켜 준다.
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}
    
    response  = requests.get(url, params=params, headers=headers).content
    # response 된 html doc를 BeautifulSoup로 파싱(변환) 한다.
    html_soup = BeautifulSoup(response, 'html.parser')
    
    return html_soup

In [7]:
def fn_get_page(page_num):
    
    ## Request Url & Params
    page_url = target_url.get('domain') + target_url.get('recipe_list') # Ex : https://www.10000recipe.com + /recipe/list.html 
    page_params['page']     = page_num                                  # 스크래핑 페이지 번호
    page_params['sequence'] = 0                                         # 스크래핑 페이지의 recipe 순번
    
    print('**************************************************************')
    print('page url : ', page_url,  '\npage params : ', page_params)
    print('**************************************************************')
    
    # beautifulsoup로 html doc return
    html_page_soup = fn_get_html(page_url, page_params)
    

    ## Get Pagenation
    # page 정보 스크래핑 및 다음 페이지 존재여부 확인 가능
    list_pagenation_tag = html_page_soup.find('ul', {'class' : 'pagination'})
    
    # attrs='aria-label' : None  => 속성(Attribute) aria-label이 None(없는)  것들만 찾는다.
    list_page_tag       = list_pagenation_tag.find_all('a', attrs={'aria-label': None})
    
    # attrs='aria-label' : None  => 속성(Attribute) aria-label이 Next인 것들만 찾는다.
    list_page_next      = list_pagenation_tag.find_all('a', attrs={'aria-label': 'Next'})
    list_page_num       = [x.get_text() for x in list_page_tag] # 페이지 리스트 태그 존재
    
    ## Get Recipe Url List
    # 스크래핑 주 목적인 레시피의 url 리스트를 가져온다.
    list_recipe_tag = html_page_soup.find_all('a', attrs={'class' : 'common_sp_link'})
    list_recipe_url = [x.attrs['href'] for x in list_recipe_tag]
  
    ## Progress Bar && Recipe Scriping
    # 프로그래스 바를 보기 위해 리스트를 tqdm_notebook으로 감싼다.
    pbar = tqdm_notebook(list_recipe_url)
    
    # 데이터 가공시 error가 나더라도 계속 진행한다.
    for f_recipe_url in pbar:
        try:
            # 현재 페이지의 레시피의 순서
            page_params['sequence'] = page_params['sequence'] + 1
            
            #프로그래스 바의 설명 글을 위한 string Format
            str_desc = 'type:{0} page : {1}, seq: {2}'.format(
                page_params.get('cat4_name'),
                page_params.get('page'), 
                page_params.get('sequence')
            )

            #프로그래스 바 실행
            pbar.set_description(str_desc)
            
            #레시피 List안에 담긴 url들을 하나씩 해당 Url로 접근하여 스크래핑 진행
            fn_get_recipe(f_recipe_url)
        except:
            continue
            
    clear_output(wait=True)
    
    if (len(list_page_next) > 0):
        return True
    else:
        return False
    

In [8]:
# Recipe 상세 페이지의 스크래핑 및 데이터 저장 Function
def fn_get_recipe(recipe_url):
    
    ## Request Url & Params
    target_url['recipe'] = recipe_url                                 # Ex : /recipe/6912734
    recipe_url = target_url.get('domain') + target_url.get('recipe')  # Ex : https://www.10000recipe.com + /recipe/6912734
    
    
    # 데이터 저장을 위한 dict
    data_params = {
        'page_num'     : page_params.get('page'),         # 페이지 번호 
        'sequence'     : page_params.get('sequence'),     # recipe 순번
        'cooking_type' : page_params.get('cat4_name'),    # 요리 분류 Ex 밑반찬, 
        'title'        : '',                              # 
        'ingredient'   : '',
        'source'       : '',
        'uri'          : recipe_url
    }
    
    # beautifulsoup로 html doc return
    html_recipe_soup = fn_get_html(recipe_url)

    ## Titile
    data_params['title'] = html_recipe_soup.find('div', {'class' : 'view2_summary st3'}).find('h3').get_text()
     
    # 데이터 가공시 error가 나더라도 존재하는 데이터만 가지고 저장을 진행하고 다음 프로세스를 진행한다.
    try:
        
        ## Recipe
        recipe_area = html_recipe_soup.find('div', {'id' : 'divConfirmedMaterialArea'})

        ## Recipe - span 제거 
        span_elements = recipe_area.find_all('span')
        for span in span_elements:
            span.decompose()

        ## Recipe And Source List
        list_recipe = [x.get_text(strip=True) for x in recipe_area.find_all('ul')[0].find_all('li')]
        data_params['ingredient'] = ','.join(list_recipe)
        
        if(len(recipe_area.find_all('ul')) > 1):
            list_source = [x.get_text(strip=True) for x in recipe_area.find_all('ul')[1].find_all('li')]
            data_params['source'] = ','.join(list_source)
    finally:
        fn_save_csv(data_params)
        fn_insert_sql(data_params)

In [17]:
while True:

    # 현재 돌릴 페이지 저장.
    next_page_number = page_params['page']

    # 스크래핑 진행하고 나서 다음 페이지 존재여부 확인.
    bool_next_page_existence = fn_get_page(next_page_number)

    # 다음 페이지 존재 시
    if(bool_next_page_existence):
         page_params['page'] = page_params['page'] +1
    else:
    # 다음 페이지 존재 하지 않을 시
         break
    

**************************************************************
page url :  https://www.10000recipe.com/recipe/list.html 
page params :  {'order': 'reco', 'cat4': '66', 'cat4_name': '빵', 'page': 162, 'sequence': 0}
**************************************************************


KeyboardInterrupt: 